Object Detection 분야에서 Scale-Invariant 는 아주 중요한 과제라고 한다. 

예전에는 다양한 크기의 물체를 탐지하기 위해 이미지 자체의 크기를 리사이즈 하면서 물체를 찾았다고 한다. 

이런 작업은 메모리 및 시간 측면에서 비효율적이기 때문에

Feature Pyramid Network (FPN) 이라는 방법이 등장하게 되었다. 

먼저 FPN이 왜 나왔는지 살펴보기 위해 기존의 방법들 부터 천천히 보고자 한다. 


#### (a) Featurized Image Pyramid
<img src='./imgs/fpn1.png' width=400>

hand-engineered feature를 사용하여 다양한 크기에서 feature map을 추출합니다.

각 level에서 독립적으로 Feature를 추출한 후 예측하는 방식이기 때문에 광범위한 객체를 탐지할 수 있으나, 연산량과 시간 관점에서 매우 비효율적이며, 현실에 적용하기가 어렵습니다.

#### (b) Single Feature Map
<img src='./imgs/fpn2.png' width=400>

이 방법은 컨볼루션 레이어가 스케일 변화에 로버스트 하기 때문에 

컨볼루션 레이어를 통해서 특징을 압축하는 방식이다. 하지만 고정된 크기의 입력만을 받을 수 있고, 다양한 크기에서 feature를 나타낼 수 없기 때문에(multi-scale feature representation) 성능이 부족하여 한번에 특징을 압축하여 마지막에 압축된 특징만을 사용하기 때문에 성능이 떨어진다는 평가가 있다. 


#### (c) Pyramidal Feature Hierarchy
<img src='./imgs/fpn3.png' width=400>

SSD(Single Shot multibox Detector)에서 사용하는 방식입니다. Convolution 연산을 수행할수록 feature map size가 작아진다는 점을 이용하여 서로 다른 크기의 feature map을 이용하여 예측을 수행하게 됩니다. 각 Pyramid level에서 독립적으로 feature를 추출하여 예측하지만, Convolution 연산을 수행할수록 Semantic feature는 더욱 강해지지만 이미지의 해상도가 떨어져 작은 Object를 감지하기가 어렵다는 특징이 있습니다.

#### (d) Feature Pyramid Network
<img src='./imgs/fpn4.png' width=400>

Top-down pathway와 lateral connection을 이용하여 저해상도의 이미지와 고해상도의 이미지를 더하여 압축된 feature map에 해상도를 보완한다는 방식입니다. 역시나 3)과 마찬가지로 독립된 level에서 독립적으로 feature를 추출하여 예측합니다. 더하여 multi-scale feature representation을 효율적으로 사용한다는 장점이 있습니다.

이는 skip connection, top-down, cnn forward 에서 생성되는 피라미드 구조를 합친 형태이다. 
forward 에서 추출된 의미 정보들을 top-down 과정에서 업샘플링하여 해상도를 올리고
forward에서 손실된 지역적인 정보들을 skip connection 으로 보충해서 스케일 변화에 강인하게 되는 것이다.

 
이 FPN 모델은 Region Proposeal Network (RPN) 와 Fast R-CNN을 기반으로 한다. 
FPN 모델은 입력으로 임의의 크기의 단일 스케일 영상을 다루며, 
전체적으로 Convolutional 방식으로 비례된 크기의 특징 맵을 다중 레벨로 출력하게 된다. 
이 프로세스는 Backbone Convolutional Architecture 와는 독립적이며 ResNet을 사용하여 결과를 보여주게 된다. 


<img src='./imgs/fpn6.png' width=500>

CNNd은 구조적으로 피라미드형 bottom-up 방식의 feature를 뽑을 수 있지만 위로 갈수록 semantic(의미)은 더 강해지지만, local 정보가 부족해진다. 이러한  trade-off를 보완하기 위해 기존의 CNN구조인 bottom-up파트에서의 high-level과 low-level의 feature를 더하는 방식의 top-down & lateral connection이 제안된것이다. 

FPN은 RPN과 instance segmentation으로 활용이 가능하다.

> low-level은 피라미드의 CNN 구조에서 1, 2번째 layer이고, high-level은 CNN 구조의 말단이라고 생각하자


#### Bottom-up pathway

backbone CNN의 feed foward 계산입니다. 여기에서는 각 Layer마다 하나의 Pyramid level을 정의하고, 동일한 크기의 output feature map을 생성하는 Layer들이 있다면, 가장 마지막 Layer의 출력을 reference feature map으로 선택합니다. 가장 깊은즉, 가장 마지막의 Layer가 가장 강한 feature를 가지기 때문입니다. 입력 이미지의 메모리 공간이 커서 Pyramid에 conv1은 포함시키지 않았습니다. 각 level의 마지막 layer를 C2, C3, C4, C5라고 부릅니다.

CNN 자체가 레이어를 거치면서 피라미드 구조를 만들고 forward 를 거치면서 더 많은 의미(Semantic)를 가지게 된다. 
각 레이어마다 예측 과정을 넣어서 Scale 변화에 더 강한 모델이 되는 것이다. 

이는 스케일링 단계가 2인 여러 특징 맵으로 구성된 특징 계층을 계산한다. 
종종 동일한 크기의 출력 맵을 생성하는 레이어가 많이 있으며 이러한 레이어가 동일한 네트워크 단계에 있다. 

즉, 위로 올라가는 forward 단계 에서는 매 레이어마다 의미 정보를 응축하는 역할을 하게 된다. 
깊은 모델의 경우 가로, 세로 크기가 같은 레이어들이 여러개 있을 수 있는데 이 경우에 같은 레이어들은
하나의 단계로 취급해서 각 단계의 맨 마지막 레이어를 skip-connection 에 사용하게 된다. 

즉, 각 단계의 마지막 레이어의 출력을 특징 맵의 Reference Set 으로 선택한다. 
피라미드를 풍부하게 생성하며, 각 단계의 가장 깊은 레이어에는 가장 영향력 있는 특징이 있어야 한다. 
특히 ResNet의 경우 각 단계의 마지막 잔차 블록(Residual Block)에서 출력되는 Feature Activation을 활성화 한다.

conv2, conv3, conv3, conv5 출력에 대해 마지막 잔차 블록의 출력을 C2, C3, C4, C5 라고 표시하고
각 레이어에는 4, 8, 16, 32 픽셀의 Stride 가 설정된다. 
conv1 은 큰 메모리를 차지하기 때문에 피라미드에 포함시키지 않는다. 

 

 

#### Top-down pathway and lateral connections
<img src='./imgs/fpn5.png' width=400>

Sematic이 강한 feature map을 2배로 Upsampling 하고 Bottom-Up pathway에서 선택한 reference feature map을 더하여 해상도를 보완하는 것(손실된 지역적 정보를 보충)이 핵심입니다. 채널 수(논문에서 d=256)를 맞춰주기 위해 reference feature map에 1 × 1 Convolution 연산을 수행합니다. 마지막으로 병합한 feature map에 3 × 3 Convolution 연산을 추가하여 최종적인 feature map(P2, P3, P4, P5)을 생성합니다. 이는 Upsampling의 aliasing(신호가 변형되어 일그러짐이 생기는 현상)을 제거하기 위함입니다. 

Top-down 과정에서 매 레이어 마다 classfifier / regressor 가 적용되는데 
같은 classfifier / regressor 를 사용하기 때문에 채널이 전부 같고, 논문에서는 256 채널을 사용했다고 한다.
때문에 skip connection 을 합칠 때 1 x 1 레이어를 한번 거쳐서 채널을 알맞게 바꿔준다. 

즉, 업샘플링된 맵은 Element-wise addition 에 의해 상향식 맵과 병합되는 과정을 거친다. 
이 때 채널 수를 줄이기 위해 1 x 1 컨볼루션 레이어를 거치게 된다. 
이 프로세스는 마지막 resolution map이 생성될 때 까지 반복된다. 
반복을 시작하기 위해 C5에 1 x 1 컨볼루션 레이어를 덧붙여 가장 정교한 resolution map을 생성한다. 

마지막으로 각 합쳐진 맵에 3 x 3 컨볼루션을 추가하여 업샘플링의 앨리어싱(Aliasing) 효과를 줄이는 최종 특징 맵을 생성한다. 
생성된 특징 맵은 각각 P2, P3, P4, P5 라고 불리며, 각 동일한 Spatial Size인 C2, C3, C4, C5에 대응된다. 
 

 

1) Bottom-Up Pathway

위의 그림에서 3단계의 Level로 구성된 Bottom-UpPathway를 볼수있다.

각 Layer는 여러 층으로 이루어 질 수 있고, 이렇게 구성된 Layer를 Residual Block이라고 한다.

각 Layer(Residual Block)의 마지막 층의 출력을 각 layer의출력으로 이용한다.

여기서 사용할 출력은위의 그림과 달리 4개의 출력을 이용한다. { C2, C3, C4,C5 }

이들은 Conv2, Conv3, Conv4, Conv5 의 출력이고, 입력입미지와크기를 비교해 보면,

Stride의 크기가 { 4, 8, 16, 32 }이다. Conv1은 메모리를 너무 많이 차지해서 제외하도록 한다.

 

2) Top-Down Pathway and Lateral Connections

마치 Encoder와 Decoder에서 Decoder처럼동작한다.

Bottom-Up Pathway에서 나오는 각 Layer(ResidualBlock)의 출력을

Lateral Connection으로 넘겨받고,

상층에서 ScaleUp되어 전달되는 입력을 더해서 점차적으로 Scale을 키워가는연산을 한다.

이번 Pathway를 시작하기 직전에 Coarest Resolution Map의생성을 위해

C5다음에 1x1 FC를 집어넣는다. 

Upsapling 알고리즘은 단순히 Nearest Neighbour를사용한다.

Lateral Connection과의 ElementWise 덧셈을 위해서Lateral Connection에 1x1의 Convolution

을 사용해서 Channel의 수를 맞추어 준다.

ElementWise 덧셈 다음에는 Upsampling직후의 Aliasing Effect를 줄이기 위해

3x3 Convolution을 수행한다.

이렇게하면 {C2, C3, C4, C5}에 대응하는 {P2, P3, P4, P5}의Feature Map을 만들수 있다.

각 Layer들은 Classifer와 Regressor들을공유하기 때문에

Feature Dimension을 통일할 필요가 있고, d=256으로하였다.


## FOR RPN(Region Proposal Networks)
<img src='./imgs/fpn7.png'>

RPN에  적용하기 위하여 feature pyramid의 각 level에 RPN head를 부착시킵니다. RPN head는 3 × 3 conv 및 2개의 형제(sibling) 1 × 1 conv로 이루어져 있습니다. 각 level(P2, P3, P4, P5, P6)에서 {1 : 2, 1 : 1, 2 : 2} 3개의 aspect ratio의 anchor(크기는 각 level 별로 322, 642, 1282, 2562, 5122)를 사용하여 총 15개의 anchor를 사용했습니다. Ground Truth에 대해서 IoU가 0.7보다 크면 positive label로 지정하고, 0.3보다 낮으면 negative label로 지정했습니다. head의 parameter는 모든 feature pyramid level에서 공유됩니다. 

<img src='./imgs/fpn8.png'>

FPN을 통하여 다양한 크기의 feature map을 추출하고, 가장 해상도가 높은 feature map을 이용하여 Region Proposal 단계를 수행합니다. 이때 RPN을 사용했습니다. 각 제안된 영역을 적당한 크기의 feature map P_k에 할당하였습니다.

<img src='./imgs/fpn9.png'>

그 후, RoI Pooling 단계를 거쳐 Bounding Box regression과 Classification을 수행합니다.


### Conclusion

Top-down pathway 없이 Bottom-Up pathway 만으로 Network를 설계하면 Layer 별 Semantic 수준이 다르기 때문에 성능이 좋지 않습니다. 깊으면 깊을수록 이러한 현상이 두드러집니다. 또한 lateral connection이 없다면 Top-down pathway를 가짐으로써 높은 sematic feature를 가지고 있을 순 있지만 이미지 해상도가 크게 떨어져 지역 정보가 부정확하여 정확도가 떨어집니다. 따라서 이전 연산에서 계층별 Output feature map을 재사용하는 것이 Multi-scale feature representation에 효율적이며 정확도 향상에 좋습니다. 결론적으로 Multi-scale feature representation이 매우 중요함을 알 수 있습니다.
[출처] Feature Pyramid Networks for Object Detection|작성자 VIM HJK